In [200]:
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [201]:
with open('../data/flutter/flutter_issues_labeled.json') as json_data:
    data = json.load(json_data)

In [202]:
with open('../data/flutter/flutter_pulls_comments_1.json') as json_data:
    pulls1 = json.load(json_data)
with open('../data/flutter/flutter_pulls_comments_2.json') as json_data:
    pulls2 = json.load(json_data)
combined_pulls = []
for pull in pulls1:
    if type(pull['comments']) is list:
        combined_pulls.append(pull)
for pull in pulls2:
    if type(pull['comments']) is list:
        combined_pulls.append(pull)
with open('../data/flutter/flutter_pulls_comments.json', 'w') as json_data:
    json.dump(combined_pulls, json_data, indent=4)

In [203]:
for issue in data:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-203-cbf55b2ffc17>, line 2)

In [ ]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if issue['training_labels'][assignee] == 1:
            num_positive_labels += 1
        total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

Now filter out any issues that never end up getting a commit/PR before they are closed. Idea being that these are not 'real' issues.

In [ ]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                num_positive_labels += 1
            total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

In [ ]:
person_counts = {}
person_totals = {}
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                person_counts[assignee] = person_counts.get(assignee, 0) + 1
            person_totals[assignee] = person_totals.get(assignee, 0) + 1
person_percents = {}
for person in person_totals:
    person_percents[person] = person_counts.get(person, 0) / person_totals[person]
    
print("Number of successful tasks: ")
print(person_counts)
print("Number of total tasks: ")
print(person_totals)
print("Percentage of tasks completed: ")
print(person_percents)

In [ ]:
#TODO: compare characteristics of issues that are successfully completed.